# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Setup Cluster and Ingress

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Istio Ingress](seldon_core_setup.ipynb#Istio). Instructions [also online](./seldon_core_setup.html).

## Configure Istio

For this example we will create the default istio gateway for seldon which needs to be called `seldon-gateway`. You can supply your own gateway by adding to your SeldonDeployments resources the annotation `seldon.io/istio-gateway` with values the name of your istio gateway.

Create a gateway for our istio-ingress

In [ ]:
!kubectl create -f resources/seldon-gateway.yaml

Label our namespace so istio creates sidecars

In [ ]:
!kubectl label namespace seldon istio-injection=enabled

If you are using Minikube for your Kubernetes cluster you will need to run as root in a separte terminal:
```
minikube tunnel
```
This will allow a LoadBalancer to be simulated on your local machine. 

In [ ]:
INGRESS_HOST=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
INGRESS_PORT=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].port}'
ISTIO_GATEWAY=INGRESS_HOST[0]+":"+INGRESS_PORT[0]

In [ ]:
ISTIO_GATEWAY

## Start Seldon Core

Use the setup notebook to [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core) with Istio Ingress. Instructions [also online](./seldon_core_setup.html).

## Serve Single Model

In [ ]:
!helm install mymodel ../helm-charts/seldon-single-model

In [ ]:
!helm template ../helm-charts/seldon-single-model | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="istio",transport="grpc")
print(r)

In [ ]:
!helm delete mymodel

## Serve AB Test

In [ ]:
!helm install myabtest ../helm-charts/seldon-abtest

In [ ]:
!helm template ../helm-charts/seldon-abtest | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/myabtest-myabtest-41de5b8
!kubectl rollout status deploy/myabtest-myabtest-df66c5c

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="myabtest",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="istio",transport="grpc")
print(r)

In [ ]:
!helm delete myabtest

## Serve Multi-Armed Bandit

In [ ]:
!helm install mymab ../helm-charts/seldon-mab

In [ ]:
!helm template ../helm-charts/seldon-mab | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymab-mymab-41de5b8
!kubectl rollout status deploy/mymab-mymab-b8038b2
!kubectl rollout status deploy/mymab-mymab-df66c5c

### Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymab",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

#### REST Request

In [ ]:
r = sc.predict(gateway="istio",transport="rest")
print(r)

#### gRPC Request

In [ ]:
r = sc.predict(gateway="istio",transport="grpc")
print(r)

In [ ]:
!helm delete mymab

## Serve with Shadow

#### We'll use a pre-packaged model server but the 'shadow' flag can be set on any predictor.

In [ ]:
!pygmentize ./resources/istio_shadow.yaml

In [ ]:
!kubectl apply -f ./resources/istio_shadow.yaml

In [ ]:
!kubectl rollout status deploy/iris-default-54fcd84
!kubectl rollout status deploy/iris-shadow-446a1b8

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon",gateway_endpoint=ISTIO_GATEWAY)

In [ ]:
r = sc.predict(gateway="istio",transport="rest",shape=(1,4))
print(r)

#### The traffic should go to both the default predictor and the shadow. If desired this can be checked in istio dashboards in the same way as with the istio canary example. When shadowing only the responses from the default predictor are used.

In [ ]:
!kubectl delete -f ./resources/istio_shadow.yaml